# Sharp Ratio & Frontier

In [ ]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

%matplotlib inline

ds_yahoo = 'yahoo'

last_year = datetime.datetime.now().year - 15
start = datetime.datetime.now().replace(year=last_year)
end = datetime.datetime.now()

## Portfolio Atual

### Pegando os dados na API do Yahoo

In [ ]:
tickers = ['LB', 'M', 'DIS', 'IAU']

lb = web.DataReader(tickers[0], ds_yahoo, start, end)
m = web.DataReader(tickers[1], ds_yahoo, start, end)
dis = web.DataReader(tickers[2], ds_yahoo, start, end)
iau = web.DataReader(tickers[3], ds_yahoo, start, end)

## Normalizando o retorno diário de cada ativo do portfólio

In [ ]:
for acao_df in (lb, m, dis, iau):
    acao_df['Retorno Normalizado'] = acao_df['Adj Close'] / acao_df.iloc[0]['Adj Close']


## Alocando cada ativo com um peso específico

In [ ]:
pesos = [0.05134083, 0.02230933, 0.5678067 , 0.35854314]

for acao_df, alocacao in zip((lb, m, dis, iau), pesos):
    acao_df['Alocacao'] = acao_df['Retorno Normalizado'] * alocacao

## Posição de cada ativo no portfólio

In [ ]:
for acao_df in (lb, m, dis, iau):
    acao_df['Posicao'] = acao_df['Alocacao'] * 10000

In [ ]:
todas_posicoes = [lb['Posicao'], m['Posicao'], dis['Posicao'], iau['Posicao']]

# DF que terá colunas adicionadas como Total e Retorno Diário
portfolio_posicao = pd.concat(todas_posicoes, axis=1)

# DF que terá apenas as ações com o Adj Close column
portfolio_acoes = pd.concat(todas_posicoes, axis=1)

In [ ]:
portfolio_posicao.columns = ['Limited Brands', 'Macys', 'Disney', 'IAU']
portfolio_acoes.columns = ['Limited Brands', 'Macys', 'Disney', 'IAU']

## Desempenho da carteira

In [ ]:
portfolio_posicao['Total'] = portfolio_posicao.sum(axis=1)

In [ ]:
print('Investimento inicial:')
print(portfolio_posicao['Total'][0])
print('Investimento final:')
print(round(portfolio_posicao['Total'][-1], 2))

In [ ]:
portfolio_posicao['Total'].plot(figsize=(17,6))
plt.title('Posição total do portfolio');

In [ ]:
portfolio_acoes.plot(figsize=(17,6));

## Desempenho da Carteira Normalizada

In [ ]:
portfolio_normalizado = portfolio_acoes / portfolio_acoes.iloc[0]
portfolio_normalizado.plot(figsize=(17,6));

## Retorno diário da carteira

In [ ]:
portfolio_posicao['Retorno Diario'] = portfolio_posicao['Total'].pct_change(1)

In [ ]:
# Média de retorno diário
round_ret_medio_diario = round(portfolio_posicao['Retorno Diario'].mean() * 100, 2)
print('Essa carteira rende em média {}% diariamente'.format(round_ret_medio_diario))

In [ ]:
# Desvio Padrão do retorno diário
portfolio_posicao['Retorno Diario'].std()

In [ ]:
# Gráfico de densidade
portfolio_posicao['Retorno Diario'].plot(kind='kde', figsize=(4,5));

## Retorno Total da Carteira

In [ ]:
retorno_total_acumulado = 100 * (portfolio_posicao['Total'][-1] / portfolio_posicao['Total'][0] - 1)
round_ret = round(retorno_total_acumulado, 3)
print('Essa carteira rendeu {} %'.format(round_ret))

In [ ]:
sharpe_ratio = (portfolio_posicao['Retorno Diario'].mean()) / portfolio_posicao['Retorno Diario'].std()
sharpe_ratio

In [ ]:
sharpe_ratio_ano = (245 ** 0.5) * sharpe_ratio
sharpe_ratio_ano